### Analyzing the Correlation between Supershop Sales and Weather Patterns in Myanmar (Yangon, Mandalay, Naypyitaw): A Data-Driven Investigation

### Import Necessary Libraries


In [1]:
import logging, sys, re
import pandas as pd
import sqlite3

### Load Data

In [2]:
try:
    connection = sqlite3.connect("../data/analysis.sqlite")
    weather = pd.read_sql_query(f"SELECT * FROM weather", connection)
except sqlite3.Error as e:
    logging.error(msg=f"Error while creating SQLite DB: {e}")
    sys.exit(1)
finally:
    connection.close()

### Introduction

The retail industry, particularly supermarkets, is heavily influenced by external factors such as weather conditions. Understanding the relationship between weather patterns and Supershop sales is crucial for optimizing inventory management, pricing strategies, and marketing efforts. This project aims to explore the impact of weather on Supershop sales in three major cities in Myanmar: Yangon, Mandalay, and Naypyitaw. The significance of understanding this correlation lies in its potential to revolutionize conventional strategies related to inventory management, pricing dynamics, and targeted marketing initiatives. By unraveling the interplay between weather nuances and purchasing trends, businesses can harness the power of predictive analytics to anticipate and respond to fluctuations in demand effectively. 

The subsequent sections of this report are structured as follows: Section 2 provides a comprehensive overview and analysis of the datasets. Following this, Section 3 elucidates the adopted methodology. Subsequently, Section 4 encompasses the presentation of the project's results. Finally, Section 5 comprises discussions, and remarks, and outlines avenues for future work in this project.


### Methods:


#### Data Sources:
I had to find two datasets for this analysis, and after some searching, I came across these two datasets online, one for each of the cities: Yangon, Mandalay, and Naypyitaw, I will explain them below,


#### Supermarket Sales Data:
This dataset provides daily sales information from Supershops in the specified cities, offering a comprehensive view of sales trends for Yangon, Mandalay, and Naypyitaw city. It is a very popular open-source dataset, I have collected it from Kaggle. 
         
* Metadata URL: https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales/data
* Data URL: https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales/download?datasetVersionNumber=3
* Data Type: CSV


<table>
    <tr>
        <th> Column index</th>
        <th> Column name </th>
        <th> Description </th>
    </tr>
    <tr>
        <td> 0 </td> 
        <td> Invoice id </td> 
        <td> Invoice identification number </td> 
    </tr>
    <tr>
        <td> 1 </td>
        <td> Branch </td>
        <td> Branch (3 branches available: A, B and C) </td> 
    </tr>
    <tr>
        <td> 2 </td>
        <td> City </td>
        <td> City name </td>
    </tr>
    <tr>
        <td> 3 </td>
        <td> Customer type </td>
        <td> Type of customers. </td> 
    </tr>
    <tr>
        <td> 4 </td> 
        <td> Gender </td>
        <td> Gender </td> 
    </tr>
    <tr>
        <td> 5 </td> 
        <td> Product line </td> 
        <td> General item categorization groups  </td> 
    </tr>
    <tr>
        <td> 6 </td> 
        <td> Unit price </td> 
        <td> Price of each product in \$ </td> 
    </tr>
    <tr>
        <td> 7 </td> 
        <td> Quantity </td> 
        <td> Number of products purchased  </td> 
    </tr>
    <tr>
        <td> 8 </td> 
        <td> Tax </td>
        <td> 5\% tax fee for customer buying  </td> 
    </tr>
    <tr>
        <td> 9 </td> 
        <td> Total </td>
        <td> Total price including tax </td>
    </tr>
    <tr>
        <td> 10 </td> 
        <td> Date </td> 
        <td> Date of purchase (Record available from January 2019 to March 2019)  </td> 
    </tr>
    <tr>
        <td> 11 </td> 
        <td> Time </td> 
        <td> Purchase time (10am to 9pm) </td> 
    </tr>
    <tr>
        <td> 12 </td> 
        <td> Payment </td> 
        <td> Payment type  </td> 
    </tr>
    <tr>
        <td> 13 </td> 
        <td> COGS </td> 
        <td> Cost of goods sold </td> 
    </tr>
    <tr>
        <td> 14 </td> 
        <td> Gross margin percentage </td> 
        <td> Gross margin percentage </td> 
    </tr>
    <tr>
        <td> 15 </td> 
        <td> Gross income </td>
        <td> Gross income </td> 
    </tr>
    <tr>
        <td> 16 </td> 
        <td> Rating </td> 
        <td> Customer stratification rating (On a scale of 1 to 10) </td> 
    </tr>
    <tr>
    <caption>Table 1: Daily super market sales dataset.</caption>
</table>


#### Weather Data:
Weather data includes factors like temperature, humidity, wind speed, and precipitation. Obtained from Meteostat, this data aims to capture the meteorological conditions influencing sales.
* Metadata URL: https://dev.meteostat.net/bulk/daily.html
* Data URL: https://bulk.meteostat.net/v2/daily/48097.csv.gz
* Data Type: CSV

Datasource3: Daily Weather data for Mandalay
* Metadata URL: https://dev.meteostat.net/bulk/daily.html
* Data URL: https://bulk.meteostat.net/v2/daily/48042.csv.gz
* Data Type: CSV

Datasource4: Daily Weather data for Naypyitaw
* Metadata URL: https://dev.meteostat.net/bulk/daily.html
* Data URL: https://bulk.meteostat.net/v2/daily/VYNT0.csv.gz
* Data Type: CSV



<table>
    <tr>
        <th> column index</th>
        <th> column name </th>
        <th> description </th>
    </tr>
    <tr>
        <td> 0 </td> 
        <td> date </td> 
        <td> The date string (format: YYYY-MM-DD) </td> 
    </tr>
    <tr>
        <td> 1 </td>
        <td> tavg </td>
        <td> The average air temperature in °C </td> 
    </tr>
    <tr>
        <td> 2 </td>
        <td> tmin </td>
        <td> The minimum air temperature in °C </td>
    </tr>
    <tr>
        <td> 3 </td>
        <td> tmax </td>
        <td> The maximum air temperature in °C </td> 
    </tr>
    <tr>
        <td> 4 </td> 
        <td> prcp </td>
        <td> The daily precipitation total in mm </td> 
    </tr>
    <tr>
        <td> 5 </td> 
        <td> snow </td> 
        <td> The maximum snow depth in mm </td> 
    </tr>
    <tr>
        <td> 6 </td> 
        <td> wdir </td> 
        <td> The average wind direction in degrees (°) </td> 
    </tr>
    <tr>
        <td> 7 </td> 
        <td> wspd </td> 
        <td> The average wind speed in km/h  </td> 
    </tr>
    <tr>
        <td> 8 </td> 
        <td> wpgt </td>
        <td> The peak wind gust in km/h  </td> 
    </tr>
    <tr>
        <td> 9 </td> 
        <td> pres </td>
        <td> The average sea-level air pressure in hPa </td>
    </tr>
    <tr>
        <td> 10 </td> 
        <td> tsun </td> 
        <td> The daily sunshine total in minutes (m)  </td> 
    </tr>
    <caption>Table 2: Summery of Yagon, Mandalay, Naypyitaw dataset.</caption>
</table>




The choice of the Supershop sales dataset is grounded in its direct relevance to the main project question: "How does weather affect Supershop sales?" This data source allows us to establish a clear link between consumer purchasing behavior and meteorological conditions.
The inclusion of daily weather data for each city is crucial for contextualizing the external factors that may influence Supershop sales. By examining these weather datasets alongside sales data, we aim to uncover correlations and quantify the impact of various weather parameters on retail performance.
In essence, the selected datasets align with the project's objectives, providing a robust foundation for investigating the intricate relationship between Supershop sales and weather patterns in Yangon, Mandalay, and Naypyitaw.




### Data Pipeline:
To ensure data accuracy and consistency, a robust data pipeline was established. Raw data underwent thorough cleaning procedures, addressing missing values, outliers, and inconsistencies. The data were then transformed into a standardized format suitable for analysis. Integration of multiple datasets was performed to correlate air quality measurements with meteorological factors.



### Data Cleaning/Transformation:
Data cleaning involved imputing missing values using appropriate statistical methods and removing outliers to prevent skewed analysis. The transformation process included converting timestamps into a uniform format and aggregating data on a daily basis to facilitate temporal analysis. Normalization techniques were applied to standardize diverse data units.



### Challenges Encountered:
Several challenges were encountered during the data engineering process, such as dealing with sensor malfunctions, inconsistent data reporting intervals, and aligning disparate datasets. Rigorous quality control measures were implemented to mitigate these challenges and ensure the reliability of the results.


## Result: 


The results are presented in various formats, including tables, diagrams, and figures. Air quality indices over time, correlation matrices between pollutants and meteorological variables, and spatial distribution maps of pollutant concentrations are included. However, this section refrains from interpretation to maintain a focus on raw data representation.



## Discussion/Conclusions: 


### Interpretation of Results:

Upon analysis, a clear correlation emerges between certain meteorological factors and air quality. For example, elevated temperatures and stagnant wind conditions coincide with higher pollutant concentrations. Additionally, specific areas within the city exhibit consistently poorer air quality, suggesting localized sources of pollution.



### Limitations:

Despite the thorough data engineering process, limitations exist. The spatial density of monitoring stations may lead to data gaps in certain regions. The absence of real-time data poses challenges in capturing dynamic pollution events. Additionally, the complexity of urban environments introduces confounding factors that may not be fully accounted for in this analysis.



### Outlook to Future Work:

Future work should focus on expanding the spatial coverage of monitoring stations, integrating real-time data streams, and employing advanced machine-learning techniques for predictive modeling. Collaboration with urban planners and policymakers is essential to translate these findings into effective air quality management strategies.

In conclusion, this data-driven analysis lays the foundation for a comprehensive understanding of urban air quality dynamics. While providing valuable insights, it also highlights the need for continuous monitoring, improved data granularity, and interdisciplinary collaboration to address the multifaceted challenge of urban air pollution.

